# Smart Possum

**Deliverable** An application that reduces fatalities and serious injuries from vehicle accidents by raising driver awareness of potential dangers and recommending appropriate responses. Dangers could be predicted by road function (interstate, collector, etc.), area (urban vs. rural), configuration (4-way stop, right hand curve, etc).

**Scope**  United States only. Ignores well known factors, such as driving under the influence, speeding, etc. to focus on environmental and behavioral factors.

**Data** Detailed accident data from the Fatality Accident Reporting System FARS managed by the U.S. National Highway Traffic Safety Administration NHTSA. Data are available for the years 1975-2014 as dBase and SAS files.

### Import and Setup

In [5]:
%matplotlib inline
import bokeh
import math
import matplotlib
import numpy as np
import os
import pandas as pd
import re
import sklearn as sk
import scipy.misc
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import zipfile

from ftplib import FTP

matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

### Classes and functions

In [6]:
def clean_excel(excel_file, first_value, last_value):
    # print 'working with file {ef}'.format(ef=excel_file)
    with pd.ExcelFile(excel_file) as excel:
        try:
            df = excel.parse(str(excel.sheet_names[0]))
            # print 'trying sheet {sn}'.format(sn=str(excel.sheet_names[0])) 
            first_row = df.index[df.iloc[:,0].str.contains(first_value)==True][0]
            last_row = df.index[df.iloc[:,0].str.contains(last_value)==True][0]
        except:
            df = excel.parse(str(excel.sheet_names[1]))
            first_row = df.index[df.iloc[:,0].str.contains(first_value)==True][0]
            last_row = df.index[df.iloc[:,0].str.contains(last_value)==True][0]
            # print 'switched to sheet {sn}'.format(sn=str(excel.sheet_names[1]))
    # print 'finished parsing'
    main_df = df.iloc[first_row:last_row+1]
    extra_df = df.iloc[last_row+2:last_row+3]
    full_df = pd.concat([main_df, extra_df], ignore_index=True)
    return full_df 

In [7]:
def get_column_names(excel_file, first_value):
    with pd.ExcelFile(excel_file) as excel:
        try:
            df = excel.parse(str(excel.sheet_names[0]))
            print 'Trying {sn}'.format(sn=str(excel.sheet_names[0])) 
            first_row = df.index[df.iloc[:,0].str.contains(first_value)==True][0]
        except:
            df = excel.parse(str(excel.sheet_names[1]))
            first_row = df.index[df.iloc[:,0].str.contains(first_value)==True][0]
            print 'Switched to {sn}'.format(sn=str(excel.sheet_names[1]))
    names = df.iloc[:first_row].apply(lambda x: ''.join(str(x).strip()))
    print names
    new_names = []
    for name in names:
        new_name = re.sub('NaN\s*','', name)
        new_name = re.sub('Name: (Unnamed:)? (\d+)?, dtype: object','', new_name)
        new_name = re.sub('\d+\s{4}\s*','', new_name)
        new_name = re.sub('\\n', ' ', new_name)
        new_name = re.sub(' \d?\/ ?', '', new_name)
        new_name = re.sub(' $', '', new_name)
        new_names.append(new_name)
        new_names[0] = re.findall('\w+$', new_names[0])[0]
        new_names[len(new_names)-1] = re.sub('^TABLE [^ ]* ', '', 
                                            new_names[len(new_names)-1])
    print new_names
    return new_names

In [8]:
def clean_unicode(df_column):
    df_column = df_column.apply(lambda x: x.decode('unicode_escape').encode('ascii', 'ignore').strip())
    return df_column

### Get Data
Accident data available from ftp.nhtsa.dot.gov. Zipped DBF files (FoxPro, xBase, *etc*. database format) are in `fars\DBF` folders.

#### Retrieve from ftp

#### Fatality Analysis Reporting System FARS data

In [9]:
yrs = range(2010, 2015)

In [6]:
table_files = ('accident', 'cevent', 'person', 'vehicle')
for y in yrs:
    ftp = FTP('ftp.nhtsa.dot.gov')
    ftp.login('anonymous','jonathan.owen@msn.com')
    ftp.cwd('fars/%s/DBF' % y)
    f = 'FARS%s.zip' % y
    with open('data/%s' % f, 'wb') as outfile:
        ftp.retrbinary('RETR %s' % f, outfile.write)

#### General Estimate System GES data

In [7]:
for y in yrs:
    ftp = FTP('ftp.nhtsa.dot.gov')
    ftp.login('anonymous','jonathan.owen@msn.com')
    ftp.cwd('GES/GES{y2}'.format(y2=str(y)[2:4]))
    try:
        f = 'GES{y2}_Flatfile.zip'.format(y2=str(y)[2:4])
        with open('data/{yd}/{fn}'.format(yd=str(y), fn=f), 'wb') as outfile:
            ftp.retrbinary('RETR {fn}'.format(fn=f), outfile.write)
    except:
        f = 'GES{y4}flat.zip'.format(y4=str(y))
        with open('data/{yd}/{fn}'.format(yd=str(y), fn=f), 'wb') as outfile:
            ftp.retrbinary('RETR {fn}'.format(fn=f), outfile.write)
    ftp.close()

#### Unzip to data/year directories

In [7]:
for y in yrs:
    with zipfile.ZipFile('data/FARS%s.zip' % y, 'r') as z:
        z.extractall('/home/vagrant/SmartPossum/data/%s' % y)

#### Read tables from CSV and TXT files

In [11]:
table_files = ('accident', 'cevent', 'person', 'vehicle')

In [15]:
fars, ges = {},{}
for tf in table_files:
    frames_fars, frames_ges = [],[]
    for y in yrs:
        with open('data/csv/{yr}/{tb}.csv'\
                  .format(yr=y, tb=tf),'r') as f:
            frames_fars.append(pd.read_csv(f))
        if y == 2010:
            dt='TXT/{tb}'.format(tb=tf.upper())
        else:
            dt=tf.upper()
        try:
            with zipfile.ZipFile('data/{yd}/GES{y2}_Flatfile.zip'\
                                 .format(yd=str(y),y2=str(y)[2:4]),'r') as z: 
                frames_ges.append(pd.read_table(z.open('{tn}.TXT'\
                                                       .format(tn=dt))))
        except:
            with zipfile.ZipFile('data/{yd}/GES{y4}flat.zip'\
                                 .format(yd=str(y),y4=str(y)),'r') as z:
                frames_ges.append(pd.read_table(z.open('{tn}.TXT'\
                                                      .format(tn=dt))))
    fars[tf], ges[tf] = frames_fars, frames_ges

C:\Users\Jonathan Owen\Anaconda3\envs\py2.7\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19,20,21,22,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Jonathan Owen\Anaconda3\envs\py2.7\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Jonathan Owen\Anaconda3\envs\py2.7\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19,20,21,22,29,30,32,33,36,37,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Jonathan Owen\Anaconda3\envs\py2.7\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (49,50) have mixed types. Speci

#### Import XLS and XLSX files
#### Get file names from directory and create dataframe of details

In [16]:
excel_path = 'data/xlsx'
excel_files = os.listdir(excel_path)
excels = pd.DataFrame(data=excel_files, columns=['filename'])
file_year = []
source_table = []
description = []
full_path = []
for e in excels.filename:
    file_year.append(re.findall('(\d{4}).xlsx?$', e)[0])
    source_table.append(re.findall('_([^_]*)_\d{4}.xlsx?$', e)[0])
    description.append(re.findall('^(\w+)_[^_]*_\d{4}.xlsx?$', e)[0])
    full_path.append('{ep}/{fn}'.format(ep=excel_path, fn=e))
excels['year'] = file_year
excels['source_table'] = source_table
excels['description'] = description
excels['full_path'] = full_path

#### Create dictionary of data frames for imported data

In [18]:
excel_dict = {}
for i in range(excels.shape[0]):
    i_df = clean_excel(excels.full_path[i], 'Alabama', 'Wyoming')
    i_df['FILE_YEAR'] = excels.year[i]
    df_name =  excels.source_table[i]
    if excel_dict.has_key(df_name):
        frames = [i_df, excel_dict[df_name]]
        excel_dict[df_name] = pd.concat(frames) 
    else:
        excel_dict[df_name] = i_df

#### Melt vehicle miles table

In [19]:
vm2_full = excel_dict['vm2']
road_function_upper = ['INTERSTATE', 'FREEWAY OR EXPRESSWAY',
                      'OTHER PRINCIPAL ARTERIAL', 'MINOR ARTERIAL',
                      'MAJOR COLLECTOR', 'MINOR COLLECTOR', 'LOCAL']
road_function_only = []
for rf in road_function_upper:
    rf = rf.lower()
    rf = re.sub(' ', '_', rf)
    road_function_only.append(rf)
for i in range(1, 8):
    df_rural = vm2_full.loc[:,['FILE_YEAR', 'Unnamed: 0', 'Unnamed: {n}'.format(n=i)]]
    df_rural.columns = ['year', 'state', 'vehicle_miles']
    df_rural['road_area'] = 'rural'
    df_rural['road_function_only'] = road_function_only[i-1]
    df_urban = vm2_full.loc[:,['FILE_YEAR', 'Unnamed: 0', 'Unnamed: {n}'.format(n=i+8)]]
    df_urban.columns = ['year', 'state', 'vehicle_miles']
    df_urban['road_area'] = 'urban'
    df_urban['road_function_only'] = road_function_only[i-1]
    
    if i == 1:
        vm2_melt = pd.concat([df_rural, df_urban], ignore_index=True, axis=0)
    else:
        vm2_melt = pd.concat([vm2_melt, df_rural, df_urban], ignore_index=True, axis=0)
vm2_melt['year'] = pd.to_numeric(vm2_melt['year'])
vm2_melt['state'] = clean_unicode(vm2_melt['state'])
vm2_melt['road_area'] = clean_unicode(vm2_melt['road_area'])
vm2_melt['road_function_only'] = clean_unicode(vm2_melt['road_function_only'])
road_function_xref = vm2_melt.road_function_only.copy()
vm2_melt['road_function_xref'] = road_function_xref
vm2_melt.loc[((vm2_melt['road_function_xref'] == 'minor_collector')|(vm2_melt['road_function_xref'] == 'major_collector'))&
         (vm2_melt['road_area'] == 'urban'),'road_function_xref'] = 'collector'
vm2 =vm2_melt.groupby(['state', 'year', 'road_area','road_function_xref'],as_index=False)['vehicle_miles'].sum()

#### Read geographic location code GLC table

In [22]:
with open('data/GLCs_for_the_USA_and_DC.txt', 'r') as glc_file:
    glc = pd.read_table(glc_file)

#### Read road function code table

In [24]:
with open('data/Road_Function_Codes.txt', 'r') as road_file:
    roads = pd.read_table(road_file, header=None, names=['road_fnc', 'road_function_full'])
roads['road_function_full'] = clean_unicode(roads['road_function_full'])
roads['road_area'] = roads.road_function_full.str.split('-',1).str[0]
roads['road_function_only'] = roads.road_function_full.str.split('-',1).str[1]
roads['road_area'] = roads['road_area'].str.lower()
roads['road_function_only'] = roads['road_function_only'].str.lower()
roads['road_function_only'] =  roads['road_function_only'].str.replace('-| ', '_')
roads['road_function_only'] =  roads['road_function_only'].str.replace('local(_\w+)*', 'local')
roads['road_function_only'] =  roads['road_function_only'].str.replace('unknown(_\w+)*', 'unknown')
roads['road_function_only'] =  roads['road_function_only'].str.replace('(\w+_)*interstate', 'interstate')
roads['road_function_only'] =  roads['road_function_only'].str.replace('(\w+_)*freeways_or_expressways', 'freeway_or_expressway')
roads['road_function_only'] =  roads['road_function_only'].str.replace('(\w+_)*interstate', 'interstate')
roads['road_function_only'] =  roads['road_function_only'].str.replace('(\w+_)?principal_arterial(_\w+)?', 
                                                                       'other_principal_arterial')
roads.loc[roads['road_function_only'].isnull(),'road_function_only'] = 'unknown'

#### Join accident table with vehicle miles

In [25]:
accidents = pd.concat(fars['accident'])

In [26]:
new_index=range(accidents.shape[0])
accidents['new_index']=new_index
accidents.set_index('new_index', drop=True, append=False, inplace=True)

In [27]:
"""
        Clean up accidents reports for STATE, ROAD_FNC. YEAR combinations with 
        zero vehicle miles in vm2 table.
        Original ROAD_FNC code replaced by another with same road_area and 
        road_function value when accidents and vm2 tables are merged.
        
        example: ROAD_FNC code 5 (Rural-Minor Collector)
                 replaced by
                 ROAD_FNC code 4 (Rural-Major Collector)
"""

for i in range(accidents.shape[0]-1):
    if accidents.ix[i,'STATE'] in [16, 35, 2, 19, 38] and accidents.ix[i,'ROAD_FNC']==12:
            accidents.ix[i, 'ROAD_FNC'] = 13  
    if accidents.ix[i,'STATE']==38 and accidents.ix[i,'ROAD_FNC']==5:
            accidents.ix[i, 'ROAD_FNC'] = 4
    if accidents.ix[i,'STATE']==15 and accidents.ix[i,'ROAD_FNC']==1:
            accidents.ix[i, 'ROAD_FNC'] = 2 

In [28]:
accidents_roads = pd.merge(accidents, roads,
                           how='left',
                           left_on='ROAD_FNC',
                           right_on='road_fnc')
accidents_roads_glc = pd.merge(accidents_roads, glc, 
                               how='left',
                               left_on=('STATE',
                                        'CITY',
                                        'COUNTY'), 
                               right_on=('State Code',
                                         'City Code',
                                         'County Code'))
accidents_roads_glc['State Name'] = accidents_roads_glc['State Name'].str.title()
accidents_roads_glc['road_area'] = accidents_roads_glc['road_area'].str.lower()
accidents_roads_glc['road_function_only'] = accidents_roads_glc['road_function_only'].str.lower()
accidents_vm2 = pd.merge(accidents_roads_glc, vm2_melt,
                         how='left',
                        left_on=('State Name',
                                 'YEAR',
                                 'road_area',
                                 'road_function_only'),
                        right_on=('state',
                                  'year', 
                                  'road_area',
                                  'road_function_xref'))

In [29]:
accidents_select = ['ARR_HOUR', 'ARR_MIN', 'CF1', 'CF2', 'CF3', 
                    'DAY', 'DAY_WEEK', 'DRUNK_DR', 'FATALS', 'HARM_EV', 
                   'HOSP_HR', 'HOSP_MN', 'HOUR', 'LATITUDE', 'LGT_COND', 
                   'LONGITUD', 'MAN_COLL', 'MILEPT', 'MINUTE', 'MONTH', 
                   'NHS', 'NOT_HOUR', 'NOT_MIN', 'PEDS', 'PERMVIT', 
                   'PERNOTMVIT', 'PERSONS', 'PVH_INVL', 'RAIL', 'RELJCT1', 
                   'RELJCT2', 'REL_ROAD', 'ROUTE', 'SCH_BUS','SP_JUR',
                   'TWAY_ID', 'TWAY_ID2', 'TYP_INT', 'VE_FORMS', 'VE_TOTAL', 
                   'WEATHER', 'WEATHER1', 'WEATHER2', 'WRK_ZONE', 'YEAR', 
                   'STATE', 'ST_CASE','State Abbreviation', 'state', 'CITY', 
                    'COUNTY', 'City Name/County Name', 'ROAD_FNC', 'road_area', 
                   'road_function_xref', 'vehicle_miles']

In [30]:
accidents_rough = accidents_vm2[accidents_select]

In [31]:
accidents_by_vm = accidents_rough.groupby(['YEAR', 
                                           'state', 
                                           'road_area', 
                                           'road_function_xref'] 
                                           , as_index=False).agg(
                                          {'vehicle_miles': lambda x: sum(x)/sum(x>0),
                                           'FATALS': sum})

In [32]:
"""
        Multiply fatalities total FATALS by 0.5 when road_area=urban and 
        road_function_xref=collector because each report is counted twice.
        
"""

accidents_by_vm.loc[(accidents_by_vm['road_area']=='urban') & 
                    (accidents_by_vm['road_function_xref']=='collector'),
                    'FATALS'] = accidents_by_vm['FATALS']*50/100

In [33]:
accidents_by_vm['rate'] = accidents_by_vm['FATALS']/(accidents_by_vm['vehicle_miles']+(accidents_by_vm['vehicle_miles']==0))

In [34]:
road_function_set = set(accidents_by_vm.road_function_xref)
road_function_groups = pd.DataFrame({'road_function_xref':list(road_function_set),
                                     'road_function': ['collector',
                                                       'arterial',
                                                       'arterial', 
                                                       'collector',
                                                       'collector',
                                                       'arterial',
                                                       'local',
                                                       'arterial']})

In [35]:
accidents_by_vm = pd.merge(accidents_by_vm, road_function_groups,
                        left_on=('road_function_xref'),
                        right_on=('road_function_xref'))

In [36]:
"""
        Artificial record added for kde plot aesthetics.
"""

accidents_by_vm.loc[accidents_by_vm.shape[0]] = [2015, 
                                                 'Puerto Rico', 
                                                 'rural',
                                                 'local', 
                                                 20001, 
                                                 0,
                                                 0, 
                                                 'local']

In [37]:
"""
        Check accidents reported against zero vehicle miles.
        There should be none after clean up by reassigning ROAD_FNC.
"""

accidents_by_vm.loc[accidents_by_vm['vehicle_miles']==0,]

Empty DataFrame
Columns: [YEAR, state, road_area, road_function_xref, vehicle_miles, FATALS, rate, road_function]
Index: []

In [38]:
accidents_by_vm.rate.max()

0.20942890145481521

In [39]:
accidents_by_vm['log_vehicle_miles']=np.log(accidents_by_vm['vehicle_miles'])
accidents_by_vm['log_fatalities']=np.log(accidents_by_vm['FATALS'])

#### save accidents_by_vm

In [41]:
with open('save/accidents_by_vm.csv', 'w') as avm:
    accidents_by_vm.to_csv(avm)

#### Unzip and read GES tables

In [42]:
"""for y in yrs:
    frames = []
    try:
        with zipfile.ZipFile('data/{yd}/GES{y2}_Flatfile.zip'.format(yd=str(y), y2=str(y)[2:4]), 'r') as z:
            frames.append(pd.read_table(z.open()))
            vehicles = pd.concat(frames)
    except:
        with zipfile.ZipFile('data/{yd}/GES{y4}flat.zip'.format(yd=str(y), y4=str(y), 'r') as z:"""

"for y in yrs:\n    frames = []\n    try:\n        with zipfile.ZipFile('data/{yd}/GES{y2}_Flatfile.zip'.format(yd=str(y), y2=str(y)[2:4]), 'r') as z:\n            frames.append(pd.read_table(z.open()))\n            vehicles = pd.concat(frames)\n    except:\n        with zipfile.ZipFile('data/{yd}/GES{y4}flat.zip'.format(yd=str(y), y4=str(y), 'r') as z:"

## Graphs

### Fatalities vs Vehicle Miles

In [43]:
kde_cmap = sns.choose_cubehelix_palette(as_cmap=True)

In [44]:
with sns.axes_style("dark"):
    g = sns.FacetGrid(data=accidents_by_vm,
                      row='road_function',
                      col='road_area',
                      size=4)
g.set(xlim=(0,15000), ylim=(0,150))
g.map(sns.kdeplot, 
      "vehicle_miles", 
      "FATALS",
      cmap=kde_cmap,
      n_levels=75,
      shade=True)

In [45]:
with sns.axes_style("dark"):
    g = sns.FacetGrid(data=accidents_by_vm,
                      row='road_function',
                      col='road_area',
                      hue='road_function_xref',
                      palette="Blues_d",
                      size=4)
#g.set(xlim=(0,15000), ylim=(0,7))
g.map(sns.regplot, 
      'log_vehicle_miles', 
      'log_fatalities',
     robust=True)

C:\Users\Jonathan Owen\Anaconda3\envs\py2.7\lib\site-packages\numpy\lib\function_base.py:3569: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


## Prediction

### Classes

In [46]:
"""class CreateDicts(sk.base.BaseEstimator, sk.base.TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        self.dict_list = []
        for x_item in X:
            item_dict = {}
            if isinstance(x_item, basestring):
                item_dict[x_item] = 1
            else:
                for item_list in x_item:
                    item_dict[item_list] = 1
            self.dict_list.append(item_dict)
        return self.dict_list """

'class CreateDicts(sk.base.BaseEstimator, sk.base.TransformerMixin):\n    def fit(self, X, y=None):\n        return self\n    \n    def transform(self, X):\n        self.dict_list = []\n        for x_item in X:\n            item_dict = {}\n            if isinstance(x_item, basestring):\n                item_dict[x_item] = 1\n            else:\n                for item_list in x_item:\n                    item_dict[item_list] = 1\n            self.dict_list.append(item_dict)\n        return self.dict_list '

#### `ModelTransformer`()

In [ ]:
class ModelTransformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, *args, **kwargs):
        self.model.fit(*args, **kwargs)
        return self

    def transform(self, X, **transform_params):
        return pd.DataFrame(self.model.predict(X))

### Poisson regression of `FATALS` on `vehicle_miles`

In [70]:
endog = accidents_by_vm['FATALS']
exog = sm.add_constant(accidents_by_vm['vehicle_miles'], prepend=False)

In [71]:
glm_poisson = sm.GLM(endog, exog, family=sm.families.Poisson())
poisson = glm_poisson.fit()

In [72]:
print poisson.summary()

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 FATALS   No. Observations:                 2783
Model:                            GLM   Df Residuals:                     2781
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -72349.
Date:                Mon, 15 Aug 2016   Deviance:                   1.3040e+05
Time:                        16:32:44   Pearson chi2:                 1.68e+05
No. Iterations:                     9                                         
                    coef    std err          z      P>|z|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
vehicle_miles   4.74e-05   1.45e-07    326.945      0.000      4.71e-05  4.77e-05
const             3.6906      0.003   1221.

#### Simulated data from Poisson regression model

In [99]:
rate_coef = poisson.params[0]
const = poisson.params[1]
print rate_coef, const

4.74027570798e-05 3.69058112792


In [86]:
models_sim = pd.concat([accidents_by_vm[['YEAR',
                                         'state',
                                         'road_area',
                                         'road_function_xref',
                                         'vehicle_miles',
                                         'FATALS',
                                         'road_function']
                                       ].copy()]*100)

`---------|---------|---------|------- 80 characters -------|---------|----------`

In [87]:
models_sim['poisson_lambda'] = const + models_sim.loc[:,'vehicle_miles']*rate_coef

In [88]:
models_sim.shape

(278300, 8)

In [89]:
np.random.seed(42)
models_sim['poisson_FATALS'] = np.random.poisson(models_sim.loc[:,'poisson_lambda'])

In [90]:
models_sim.poisson_FATALS.max()

19

In [80]:
"""miles = 1000
counts = ((miles*rate_coef)**fatality_data)*np.exp(-1*miles*rate_coef)/scipy.misc.factorial(fatality_data)
dist_1000 = pd.DataFrame({'fatalities': fatality_data,
                           'counts': counts})"""

"miles = 1000\ncounts = ((miles*rate_coef)**fatality_data)*np.exp(-1*miles*rate_coef)/scipy.misc.factorial(fatality_data)\ndist_1000 = pd.DataFrame({'fatalities': fatality_data,\n                           'counts': counts})"

In [81]:
"""fatalities = fatality_data * len(exposure_data)"""

'fatalities = fatality_data * len(exposure_data)'

In [82]:
"""poisson_view = pd.DataFrame({'vehicle_miles': exposures,
                             'fatalities': fatalities})"""

"poisson_view = pd.DataFrame({'vehicle_miles': exposures,\n                             'fatalities': fatalities})"

In [83]:
"""poisson_view['counts'] = (poisson_view['vehicle_miles']*rate_coef)**poisson_view['fatalities'] *\
                          np.exp(-1*poisson_view['vehicle_miles']*rate_coef)/\
                          scipy.misc.factorial(poisson_view['fatalities'])"""

"poisson_view['counts'] = (poisson_view['vehicle_miles']*rate_coef)**poisson_view['fatalities'] *                          np.exp(-1*poisson_view['vehicle_miles']*rate_coef)/                          scipy.misc.factorial(poisson_view['fatalities'])"

In [92]:
p = sns.kdeplot(models_sim.vehicle_miles,
                models_sim.poisson_FATALS,
                cmap=kde_cmap,
                n_levels=75,
                shade=True)
p.set(xlim=(0,15000), ylim=(0,15))

[(0, 15), (0, 15000)]

In [96]:
glm_nbinom = sm.GLM(endog, exog, family=sm.families.NegativeBinomial())
nbinom = glm_nbinom.fit()

In [97]:
print(nbinom.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 FATALS   No. Observations:                 2783
Model:                            GLM   Df Residuals:                     2781
Model Family:        NegativeBinomial   Df Model:                            1
Link Function:                    log   Scale:                    1.0590450906
Method:                          IRLS   Log-Likelihood:                -13444.
Date:                Mon, 15 Aug 2016   Deviance:                       2797.4
Time:                        16:59:15   Pearson chi2:                 2.95e+03
No. Iterations:                    31                                         
                    coef    std err          z      P>|z|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
vehicle_miles     0.0001   2.74e-06     41.451      0.000         0.000     0.000
const             3.2417      0.024    133.

In [98]:
vm_coef = nbinom.params[0]
alpha = nbinom.params[1]
print vm_coef, alpha

0.000113375442865 3.24173799125


## Appendix

### Data views

#### `fars` dictionary

In [ ]:
fars

#### Vehicle miles and registration data
Directory of .XLS and .XLSX Excel files exported from US Department of Transportation

In [917]:
excels

filename  year source_table  \
0        vehicle_reg_by_state_mv1_2014.xlsx  2014          mv1   
1   vehicle_miles_distribution_vm4_2010.xls  2010          vm4   
2     lane_length_by_function_hm60_2013.xls  2013         hm60   
3        vehicle_reg_by_state_mv1_2011.xlsx  2011          mv1   
4    vehicle_miles_by_function_vm2_2010.xls  2010          vm2   
5    vehicle_miles_by_function_vm2_2011.xls  2011          vm2   
6      road_miles_by_function_hm20_2010.xls  2010         hm20   
7   vehicle_miles_by_function_hm44_2012.xls  2012         hm44   
8      road_miles_by_function_hm20_2011.xls  2011         hm20   
9     lane_length_by_function_hm60_2011.xls  2011         hm60   
10  vehicle_miles_distribution_vm4_2013.xls  2013          vm4   
11    lane_length_by_function_hm60_2014.xls  2014         hm60   
12   vehicle_miles_by_function_vm2_2013.xls  2013          vm2   
13  vehicle_miles_by_function_hm44_2011.xls  2011         hm44   
14       vehicle_reg_by_state_mv1_2013.xlsx  2013          mv1   
15  vehicle_miles_distribution_vm4_2014.xls  2014          vm4   
16     road_miles_by_function_hm20_2014.xls  2014         hm20   
17   vehicle_miles_by_function_vm2_2014.xls  2014          vm2   
18     road_miles_by_function_hm20_2012.xls  2012         hm20   
19  vehicle_miles_by_function_hm44_2014.xls  2014         hm44   
20    lane_length_by_function_hm60_2012.xls  2012         hm60   
21     road_miles_by_function_hm20_2013.xls  2013         hm20   
22        vehicle_reg_by_state_mv1_2010.xls  2010          mv1   
23    lane_length_by_function_hm60_2010.xls  2010         hm60   
24  vehicle_miles_by_function_hm44_2013.xls  2013         hm44   
25       vehicle_reg_by_state_mv1_2012.xlsx  2012          mv1   
26   vehicle_miles_by_function_vm2_2012.xls  2012          vm2   

                   description  \
0         vehicle_reg_by_state   
1   vehicle_miles_distribution   
2      lane_length_by_function   
3         vehicle_reg_by_state   
4    vehicle_miles_by_function   
5    vehicle_miles_by_function   
6       road_miles_by_function   
7    vehicle_miles_by_function   
8       road_miles_by_function   
9      lane_length_by_function   
10  vehicle_miles_distribution   
11     lane_length_by_function   
12   vehicle_miles_by_function   
13   vehicle_miles_by_function   
14        vehicle_reg_by_state   
15  vehicle_miles_distribution   
16      road_miles_by_function   
17   vehicle_miles_by_function   
18      road_miles_by_function   
19   vehicle_miles_by_function   
20     lane_length_by_function   
21      road_miles_by_function   
22        vehicle_reg_by_state   
23     lane_length_by_function   
24   vehicle_miles_by_function   
25        vehicle_reg_by_state   
26   vehicle_miles_by_function   

                                            full_path  
0   /home/vagrant/fars/data/xlsx/vehicle_reg_by_st...  
1   /home/vagrant/fars/data/xlsx/vehicle_miles_dis...  
2   /home/vagrant/fars/data/xlsx/lane_length_by_fu...  
3   /home/vagrant/fars/data/xlsx/vehicle_reg_by_st...  
4   /home/vagrant/fars/data/xlsx/vehicle_miles_by_...  
5   /home/vagrant/fars/data/xlsx/vehicle_miles_by_...  
6   /home/vagrant/fars/data/xlsx/road_miles_by_fun...  
7   /home/vagrant/fars/data/xlsx/vehicle_miles_by_...  
8   /home/vagrant/fars/data/xlsx/road_miles_by_fun...  
9   /home/vagrant/fars/data/xlsx/lane_length_by_fu...  
10  /home/vagrant/fars/data/xlsx/vehicle_miles_dis...  
11  /home/vagrant/fars/data/xlsx/lane_length_by_fu...  
12  /home/vagrant/fars/data/xlsx/vehicle_miles_by_...  
13  /home/vagrant/fars/data/xlsx/vehicle_miles_by_...  
14  /home/vagrant/fars/data/xlsx/vehicle_reg_by_st...  
15  /home/vagrant/fars/data/xlsx/vehicle_miles_dis...  
16  /home/vagrant/fars/data/xlsx/road_miles_by_fun...  
17  /home/vagrant/fars/data/xlsx/vehicle_miles_by_...  
18  /home/vagrant/fars/data/xlsx/road_miles_by_fun...  
19  /home/vagrant/fars/data/xlsx/vehicle_miles_by_...  
20  /home/vagrant/fars/data/xlsx/lane_length_by_fu...  
2

In [916]:
excel_dict

{'hm20':    FILE_YEAR            Unnamed: 0 Unnamed: 1 Unnamed: 10 Unnamed: 11  \
 0       2013               Alabama    610.341      42.147     1164.94   
 1       2013                Alaska    1023.93           0     131.242   
 2       2013               Arizona     920.96     207.928     1491.12   
 3       2013              Arkansas     383.38      132.97      770.95   
 4       2013            California    1207.48     1471.59      7041.8   
 5       2013              Colorado    684.474     313.031      1021.7   
 6       2013           Connecticut      43.28      239.79      689.44   
 7       2013              Delaware          0       30.38       175.6   
 8       2013  District of Columbia          0      15.856     106.574   
 9       2013               Florida    748.448     571.161     3608.18   
 10      2013               Georgia     535.85      148.95     2308.91   
 11      2013                Hawaii          0       33.65     276.824   
 12      2013                 

In [1388]:
set(accidents_vm2['state'])

{nan,
 'Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming'}

In [1375]:
vm2.iloc[2:3,0].index

MultiIndex(levels=[[u'Alabama', u'Alaska', u'Arizona', u'Arkansas', u'California', u'Colorado', u'Connecticut', u'Delaware', u'Dist. of Columbia', u'Florida', u'Georgia', u'Hawaii', u'Idaho', u'Illinois', u'Indiana', u'Iowa', u'Kansas', u'Kentucky', u'Louisiana', u'Maine', u'Maryland', u'Massachusetts', u'Michigan', u'Minnesota', u'Mississippi', u'Missouri', u'Montana', u'Nebraska', u'Nevada', u'New Hampshire', u'New Jersey', u'New Mexico', u'New York', u'North Carolina', u'North Dakota', u'Ohio', u'Oklahoma', u'Oregon', u'Pennsylvania', u'Puerto Rico', u'Puerto Rico  (2)', u'Puerto Rico  2/', u'Rhode Island', u'South Carolina', u'South Dakota', u'Tennessee', u'Texas', u'Texas (2)', u'Utah', u'Vermont', u'Virginia', u'Washington', u'West Virginia', u'Wisconsin', u'Wyoming'], [2010, 2011, 2012, 2013, 2014], [u'rural', u'urban'], [u'collector', u'freeway_or_expressway', u'interstate', u'local', u'major_collector', u'minor_arterial', u'other_principal_arterial']],
           labels=[[0], 

In [1385]:
vm2

state  year road_area        road_function_xref  vehicle_miles
0     Alabama  2010     rural                 collector    1685.570000
1     Alabama  2010     rural     freeway_or_expressway       0.000000
2     Alabama  2010     rural                interstate    5744.605837
3     Alabama  2010     rural                     local    6851.415000
4     Alabama  2010     rural           major_collector    4983.954972
5     Alabama  2010     rural            minor_arterial    4800.226643
6     Alabama  2010     rural  other_principal_arterial    6503.179197
7     Alabama  2010     urban                 collector    3273.775149
8     Alabama  2010     urban     freeway_or_expressway     591.210765
9     Alabama  2010     urban                interstate    7552.172521
10    Alabama  2010     urban                     local    8526.035000
11    Alabama  2010     urban            minor_arterial    6192.350359
12    Alabama  2010     urban  other_principal_arterial    7460.653194
13    Alabama  2011     rural                 collector    1660.057960
14    Alabama  2011     rural     freeway_or_expressway     353.979683
15    Alabama  2011     rural                interstate    5715.295684
16    Alabama  2011     rural                     local    6746.612550
17    Alabama  2011     rural           major_collector    5218.468689
18    Alabama  2011     rural            minor_arterial    4668.655319
19    Alabama  2011     rural  other_principal_arterial    5994.875654
20    Alabama  2011     urban                 collector    3725.414928
21    Alabama  2011     urban     freeway_or_expressway     682.809292
22    Alabama  2011     urban                interstate    7452.542369
23    Alabama  2011     urban                     local    8591.918595
24    Alabama  2011     urban            minor_arterial    6370.889057
25    Alabama  2011     urban  other_principal_arterial    7732.852587
26    Alabama  2012     rural                 collector    1643.420154
27    Alabama  2012     rural     freeway_or_expressway       0.052023
28    Alabama  2012     rural                interstate    6060.952563
29    Alabama  2012     rural                     local    6707.893914
...       ...   ...       ...                       ...            ...
3350  Wyoming  2012     urban                interstate     499.981791
3351  Wyoming  2012     urban                     local     709.308000
3352  Wyoming  2012     urban            minor_arterial     391.811605
3353  Wyoming  2012     urban  other_principal_arterial     804.917550
3354  Wyoming  2013     rural                 collector     737.485785
3355  Wyoming  2013     rural     freeway_or_expressway       0.000000
3356  Wyoming  2013     rural                interstate    2419.707625
3357  Wyoming  2013     rural                     local     630.446250
3358  Wyoming  2013     rural           major_collector     594.442547
3359  Wyoming  2013     rural            minor_arterial     516.020657
3360  Wyoming  2013     rural  other_principal_arterial    1547.995145
3361  Wyoming  2013     urban                 collector     433.649461
3362  Wyoming  2013     urban     freeway_or_expressway      11.089155
3363  Wyoming  2013     urban                interstate     503.828320
3364  Wyoming  2013     urban                     local     715.553300
3365  Wyoming  2013     urban            minor_arterial     423.674083
3366  Wyoming  2013     urban  other_principal_arterial     775.048067
3367  Wyoming  2014     rural                 collector     738.960750
3368  Wyoming  2014     rural     freeway_or_expressway       0.000000
3369  Wyoming  2014     rural                interstate    2459.292111
3370  Wyoming  2014     rural                     local     631.709150
3371  Wyoming  2014     rural           major_collector     600.966144
3372  Wyoming  2014     rural            minor_arterial     529.535061
3373  Wyoming  2014     rural  other_principal_arterial    1577.931078
3374  Wyoming  2014     urban   

In [1247]:
roads

road_fnc                                 road_function_full road_area  \
0          1                Rural-Principal Arterial-Interstate     rural   
1          2                     Rural-Principal Arterial-Other     rural   
2          3                               Rural-Minor Arterial     rural   
3          4                              Rural-Major Collector     rural   
4          5                              Rural-Minor Collector     rural   
5          6                         Rural-Local Road or Street     rural   
6          9                                Rural-Unknown Rural     rural   
7         11                Urban-Principal Arterial-Interstate     urban   
8         12  Urban-Principal Arterial-Other Freeways or Exp...     urban   
9         13                     Urban-Other Principal Arterial     urban   
10        14                               Urban-Minor Arterial     urban   
11        15                                    Urban-Collector     urban   
12        16                         Urban-Local Road or Street     urban   
13        19                                Urban-Unknown Urban     urban   
14        99                                            Unknown   unknown   

          road_function_only  
0                 interstate  
1   other_principal_arterial  
2             minor_arterial  
3            major_collector  
4            minor_collector  
5                      local  
6                    unknown  
7                 interstate  
8      freeway_or_expressway  
9   other_principal_arterial  
10            minor_arterial  
11                 collector  
12                     local  
13                   unknown  
14                   unknown

In [42]:
accidents_by_vm.head()

YEAR    state road_area road_function_xref  vehicle_miles  FATALS  \
0  2010  Alabama     rural         interstate    5744.605837    50.0   
1  2010  Alabama     urban         interstate    7552.172521    41.0   
2  2010   Alaska     rural         interstate     858.995539    18.0   
3  2010   Alaska     urban         interstate     663.146376     9.0   
4  2010  Arizona     rural         interstate    7132.905012   103.0   

       rate road_function  log_vehicle_miles  log_fatalities  
0  0.008704      arterial           8.656017        3.912023  
1  0.005429      arterial           8.929591        3.713572  
2  0.020955      arterial           6.755764        2.890372  
3  0.013572      arterial           6.496996        2.197225  
4  0.014440      arterial           8.872474        4.634729

In [90]:
road_function_groups

road_function        road_function_xref
0     collector           major_collector
1      arterial     freeway_or_expressway
2      arterial  other_principal_arterial
3     collector                 collector
4     collector           minor_collector
5      arterial            minor_arterial
6         local                     local
7      arterial                interstate

In [269]:
accidents_by_vm

YEAR           state road_area     road_function_xref  vehicle_miles  \
0     2010         Alabama     rural             interstate    5744.605837   
1     2010         Alabama     urban             interstate    7552.172521   
2     2010          Alaska     rural             interstate     858.995539   
3     2010          Alaska     urban             interstate     663.146376   
4     2010         Arizona     rural             interstate    7132.905012   
5     2010         Arizona     urban             interstate    6064.827159   
6     2010        Arkansas     rural             interstate    4328.487400   
7     2010        Arkansas     urban             interstate    4092.658531   
8     2010      California     rural             interstate   17565.014236   
9     2010      California     urban             interstate   68206.775561   
10    2010        Colorado     rural             interstate    4149.261308   
11    2010        Colorado     urban             interstate    7488.892683   
12    2010     Connecticut     rural             interstate     711.190090   
13    2010     Connecticut     urban             interstate    9633.222350   
14    2010        Delaware     urban             interstate    1191.380320   
15    2010         Florida     rural             interstate    9459.399003   
16    2010         Florida     urban             interstate   25371.133977   
17    2010         Georgia     rural             interstate   10007.085572   
18    2010         Georgia     urban             interstate   19087.890058   
19    2010          Hawaii     urban             interstate    1746.478072   
20    2010           Idaho     rural             interstate    2203.875913   
21    2010           Idaho     urban             interstate    1271.140204   
22    2010        Illinois     rural             interstate    9065.701907   
23    2010        Illinois     urban             interstate   23052.797111   
24    2010         Indiana     rural             interstate    7143.452183   
25    2010         Indiana     urban             interstate    9362.526155   
26    2010            Iowa     rural             interstate    4857.843977   
27    2010            Iowa     urban             interstate    2591.824741   
28    2010          Kansas     rural             interstate    3192.484158   
29    2010        Kentucky     rural             interstate    6874.633623   
...    ...             ...       ...                    ...            ...   
2753  2014          Hawaii     urban  freeway_or_expressway     493.104817   
2754  2014        Illinois     urban  freeway_or_expressway    1182.801345   
2755  2014          Kansas     urban  freeway_or_expressway    1819.782482   
2756  2014        Kentucky     urban  freeway_or_expressway     911.672402   
2757  2014       Louisiana     urban  freeway_or_expressway     506.662705   
2758  2014        Maryland     urban  freeway_or_expressway    6065.602046   
2759  2014   Massachusetts     urban  freeway_or_expressway    5887.247550   
2760  2014        Michigan     urban  freeway_or_expressway    6225.103841   
2761  2014       Minnesota     urban  freeway_or_expressway    3568.396581   
2762  2014        Missouri     urban  freeway_or_expressway    4824.310597   
2763  2014        Nebraska     urban  freeway_or_expressway     966.361543   
2764  2014          Nevada     urban  freeway_or_expressway    1606.205166   
2765  2014   New Hampshire     urban  freeway_or_expressway    1282.000258   
2766  2014      New Jersey     urban  freeway_or_expressway   12880.562453   
2767  2014        New York     urban  freeway_or_expressway   17487.311947   
2768  2014  North Carolina     urban  freeway_or_expressway    5385.266132   
2769  2014            Ohio     urban  freeway_or_expressway    6347.968818   
2770  2014        Oklahoma     urban  freeway_or_expressway    2980.231205   
2771  2014          Oregon     urban  freeway_or_expressway    1312.725785   
2772  2014    Pennsylvania  

In [64]:
poisson_sim.head()

YEAR    state road_area road_function_xref  vehicle_miles  FATALS  \
0  2010  Alabama     rural         interstate    5744.605837    50.0   
1  2010  Alabama     urban         interstate    7552.172521    41.0   
2  2010   Alaska     rural         interstate     858.995539    18.0   
3  2010   Alaska     urban         interstate     663.146376     9.0   
4  2010  Arizona     rural         interstate    7132.905012   103.0   

  road_function  
0      arterial  
1      arterial  
2      arterial  
3      arterial  
4      arterial

### Save data

In [59]:
with open('data/lat_long.csv', 'w') as ll:
    accidents.to_csv(ll, columns=('LATITUDE', 'LONGITUD'))